In [1]:
import pandas as pd
import numpy as np
import re
import json
import matplotlib.pyplot as plt
import glob, os, csv

from langdetect import detect

import gensim
from gensim.utils import simple_preprocess
from gensim import corpora, models
from gensim.models import CoherenceModel
import ast
from pprint import pprint

import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'http', 'javascript'])

from bs4 import BeautifulSoup

import pyLDAvis
import pyLDAvis.gensim

# import stanza
# # stanza.download('en')
# nlp = stanza.Pipeline('en', processors='tokenize,mwt,pos,lemma')
import gc

In [2]:
def get_domain(url):
    url = str(url).replace('https://', '').replace('http://', '')
    result = re.findall('^(?:http?\:\/\/)?[\w\-\.]+\.onion', url)
    if len(result[0].split('.')) > 2:
        result[0] = ('.').join(result[0].split('.')[1:])
    if len(result) < 1:
        return 'NA'
    else:
        return result[0]

<>:3: DeprecationWarning: invalid escape sequence \:
<>:3: DeprecationWarning: invalid escape sequence \:
<>:3: DeprecationWarning: invalid escape sequence \:
<ipython-input-2-d3df041d8b58>:3: DeprecationWarning: invalid escape sequence \:
  result = re.findall('^(?:http?\:\/\/)?[\w\-\.]+\.onion', url)


In [3]:
# Convert html file to title and body texts
def body2text(file):
    html = open(file, "rb")
    soup = BeautifulSoup(html, "html.parser")
    body = []
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()

    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    
    for line in lines:
        if line:
            body.append(line)
    title = soup.title
    if title is None:
        title_text = None
    else:
        title_text = title.string
    html.close()
    
    return [title_text, body]

In [4]:
# convert html to string
def body2text_body(html):
    soup = BeautifulSoup(html)
    result = []
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()

    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    for line in lines:
        if line:
            result.append(line)

    return result

In [5]:
stemmer = PorterStemmer()
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text))

def preprocess(sentences):
    result = []
    
    for sent in sentences:
        lemmas = []
        # tokenize
        tokens = gensim.utils.simple_preprocess(sent)
        for token in tokens:
            lemma = lemmatize_stemming(token)
            # remove stopwords
            if lemma not in stop_words:
                lemmas.append(lemma)
        # POS tagging
        nltk_tagged = nltk.pos_tag(lemmas)
        for word, tag in nltk_tagged:
            # only keep nouns
            if tag.startswith('N'):
                result.append(word)
    return result

In [6]:
def language_detect(ls):
    text = ' '.join(ls)
    result = detect(text)
    return result

In [7]:
def get_domain_name(x, y):
    return x[y]

def custom_strip(x):
    return str(x).strip().strip('\n\t')

In [8]:
def most_frequent(List): 
    counter = 0
    num = List[0] 
      
    for i in List: 
        curr_frequency = List.count(i) 
        if(curr_frequency> counter): 
            counter = curr_frequency 
            num = i 
  
    return num 

In [9]:
date = 'feb-19-0'
df = pd.read_csv('../data/universe-'+date+'.csv')
df['body_text'] = df['body_text'].apply(ast.literal_eval)
df = df[df['status'] == 'live']
df = df.drop_duplicates(subset ='url', keep = 'first')
df['url'] = df['url'].apply(lambda x: x.replace('https://', '').replace('http://', ''))
df['domain_url'] = df['url'].apply(get_domain)
df.drop(columns=['status'], inplace=True)
df = df[['domain_url', 'url', 'title', 'body_text']]
df

,domain_url,url,title,body_text
0,facebookcorewwwi.onion,pa-in.facebookcorewwwi.onion/recover/initiate?...,ਪਾਸਵਰਡ ਭੁੱਲ ਗਏ | ਲਾਗ ਇਨ ਨਹੀਂ ਕਰ ਸਕਦੇ ਹੋ | Face...,[ਪਾਸਵਰਡ ਭੁੱਲ ਗਏ | ਲਾਗ ਇਨ ਨਹੀਂ ਕਰ ਸਕਦੇ ਹੋ | Fac...
1,facebookcorewwwi.onion,l.facebookcorewwwi.onion/l.php?u=https%3A%2F%2...,Leaving Facebook,"[Leaving Facebook, Jump toSections of this pag..."
2,facebookcorewwwi.onion,l.facebookcorewwwi.onion/l.php?u=https%3A%2F%2...,NaN,[]
3,facebookcorewwwi.onion,l.facebookcorewwwi.onion/l.php?u=https%3A%2F%2...,Leaving Facebook,"[Leaving Facebook, Jump toSections of this pag..."
4,facebookcorewwwi.onion,l.facebookcorewwwi.onion/l.php?u=https%3A%2F%2...,Leaving Facebook,"[Leaving Facebook, Jump toSections of this pag..."
...,...,...,...,...
499995,galaxy3bhpzxecbywoa2j4tg43muepnhfalars4cce3fcx...,galaxy3bhpzxecbywoa2j4tg43muepnhfalars4cce3fcx...,S3X999GHOST's wire posts : Galaxy3,"[S3X999GHOST's wire posts : Galaxy3, Galaxy3, ..."
499996,galaxy3bhpzxecbywoa2j4tg43muepnhfalars4cce3fcx...,galaxy3bhpzxecbywoa2j4tg43muepnhfalars4cce3fcx...,JohnSnow2000's files : Galaxy3,"[JohnSnow2000's files : Galaxy3, Galaxy3, Log ..."
499997,galaxy3bhpzxecbywoa2j4tg43muepnhfalars4cce3fcx...,galaxy3bhpzxecbywoa2j4tg43muepnhfalars4cce3fcx...,JohnSnow2000's bookmarks : Galaxy3,"[JohnSnow2000's bookmarks : Galaxy3, Galaxy3, ..."
499998,galaxy3bhpzxecbywoa2j4tg43muepnhfalars4cce3fcx...,galaxy3bhpzxecbywoa2j4tg43muepnhfalars4cce3fcx...,JohnSnow2000's wire posts : Galaxy3,"[JohnSnow2000's wire posts : Galaxy3, Galaxy3,..."


In [10]:
split_num = 10
size = len(df)
df_list = []

for i in range(split_num):
    temp_df = df.iloc[int(i*size/split_num):int((i+1)*size/split_num), :]
    df_list.append(temp_df)

del df
gc.collect()

1708

In [11]:
processed_df_list = []
for df in df_list:
    # group urls, titles and body texts by domain
    df = df.groupby('domain_url').agg(list)
#     df.drop(columns=['url'], inplace=True)
    df.reset_index(inplace=True)

    df['body_text'] = df['body_text'].apply(lambda x: [z for y in x for z in y])
    # Only keep unique processed texts
    df['body_text'] = df['body_text'].apply(lambda x: list(set(x)))
    
#     # preprocess
#     df['body_token'] = df['body_text'].map(preprocess)
#     df = df[df['body_token'].str.len() != 0]
#     # detect language
#     df['language'] = df['body_token'].map(language_detect)
#     df.reset_index(inplace=True)
#     df.drop(columns='index' ,inplace=True)
    processed_df_list.append(df)
    df = None
    gc.collect()

In [12]:
df_all = pd.concat(processed_df_list)
del processed_df_list
gc.collect()
df_all

,domain_url,url,title,body_text
0,22222222m4cetz5t.onion,[22222222m4cetz5t.onion],[Porn Videos - XONIONS - THE BEST ONION PORN S...,"[Anal forced, Kimmy hunter, A lot young couple..."
1,2222223lne5t7eou.onion,"[24e8111a.2222223lne5t7eou.onion, 199d3835.222...",[Bitcoin Generator Exploit - Make Free Bitcoin...,[Please send 0.0008769 BTC to 3BmKFrTChibLtfvk...
2,2222apnsa5crq7wrwe4lefkyhspz4xkkq6v6dkbrodzdgu...,[2222apnsa5crq7wrwe4lefkyhspz4xkkq6v6dkbrodzdg...,[A1 Paypal Accounts Cheap],"[We accept only BTC., Price, Price 0.0070 BTC,..."
3,2222v6snawtv7eal.onion,[2222v6snawtv7eal.onion],[A1 Paypal Accounts Cheap],"[We accept only BTC., Price, Price 0.0070 BTC,..."
4,22gcscrvwbkm4ue2.onion,[22gcscrvwbkm4ue2.onion],[AMAZON GC STORE],"[Order №2546 - Awesome service!, Amazon Gift C..."
...,...,...,...,...
51,torpress2sarn7xw.onion,"[antifa.torpress2sarn7xw.onion/xmlrpc.php?rsd,...","[nan, Errado ou Errado, como se posicionar? – ...",[Como bem sabemos em nosso país acontece nesse...
52,trustedb3yde3prf.onion,[trustedb3yde3prf.onion/signin],[Drogas Market - Sign In],"[Sign Up, (lowercase only), Reset it here, Sig..."
53,xfmro77i3lixucja.onion,[xfmro77i3lixucja.onion/book/fkrlAjw7ggf2pKLv],[Learn Python the Hard Way · A Very Simple Int...,"[By cozypumpkinmammoth, Help, Programmer's Del..."
54,xjfbpuj56rdazx4iolylxplbvyft2onuerjeimlcqwaihp...,[xjfbpuj56rdazx4iolylxplbvyft2onuerjeimlcqwaih...,"[parrot – S-Config, spear – S-Config, The Blac...",[cat /sdcard/download/91fixoverlays > /etc/ini...


In [13]:
df_all[df_all['domain_url']=='NA']

,domain_url,url,title,body_text


In [14]:
df_all['url_array'] = df_all['url'].apply(lambda x: np.array(x))
df_all['name_index'] = df_all['url_array'].apply(lambda x: np.argmin(x))
df_all['domain_name'] = df_all.apply(lambda x: get_domain_name(x['title'], x['name_index']), axis=1)
df_all['domain_name'] = df_all['domain_name'].apply(lambda x: custom_strip(x))
df_all.drop(columns=['url_array', 'name_index', 'title', 'url'], inplace=True)
df_all.rename(columns={'domain_name': 'title'}, inplace=True)
df_all = df_all[['domain_url', 'title', 'body_text']]
df_all

,domain_url,title,body_text
0,22222222m4cetz5t.onion,Porn Videos - XONIONS - THE BEST ONION PORN SI...,"[Anal forced, Kimmy hunter, A lot young couple..."
1,2222223lne5t7eou.onion,Bitcoin Generator Exploit - Make Free Bitcoins!,[Please send 0.0008769 BTC to 3BmKFrTChibLtfvk...
2,2222apnsa5crq7wrwe4lefkyhspz4xkkq6v6dkbrodzdgu...,A1 Paypal Accounts Cheap,"[We accept only BTC., Price, Price 0.0070 BTC,..."
3,2222v6snawtv7eal.onion,A1 Paypal Accounts Cheap,"[We accept only BTC., Price, Price 0.0070 BTC,..."
4,22gcscrvwbkm4ue2.onion,AMAZON GC STORE,"[Order №2546 - Awesome service!, Amazon Gift C..."
...,...,...,...
51,torpress2sarn7xw.onion,April 2016 – Mobiliza Ação,[Como bem sabemos em nosso país acontece nesse...
52,trustedb3yde3prf.onion,Drogas Market - Sign In,"[Sign Up, (lowercase only), Reset it here, Sig..."
53,xfmro77i3lixucja.onion,Learn Python the Hard Way · A Very Simple Intr...,"[By cozypumpkinmammoth, Help, Programmer's Del..."
54,xjfbpuj56rdazx4iolylxplbvyft2onuerjeimlcqwaihp...,In 2017 S will skateboard again. – S-Config,[cat /sdcard/download/91fixoverlays > /etc/ini...


In [15]:
df_all.to_csv('../data/dataset-'+date+'.csv', index=False)